In [1]:
import simpy
import datetime
import openclsim.core as core
import openclsim.model as model
import shapely
import pandas as pd

In [37]:
# ----------------------------------------------------------------------------!
class OffshoreWindFarmInstallation(object):
    '''
    --------------------------------------------------------------------------
    Description:
        A Python object used for simulating the installation process of an
        offshore wind farm.
    --------------------------------------------------------------------------
    Parameters:
        start_date: datetime.datetime 
            Start date of the construction period.
        size: int
            Number of turbines being installed.
    --------------------------------------------------------------------------
    '''
    def __init__(self, start_date: datetime.datetime, size: int, **kwargs):
        
        # Assign arguments to class attributes
        self.size = size
        self.start_date = start_date
        
        # Initialise simulation environment using the SimPy framework
        self.POSIX = start_date.timestamp() # Turn datetime object to POSIX
        self.env = simpy.Environment(initial_time=self.POSIX)

    def create_entities(self):
        
        port = dict(env=self.env, name='CHERBOURG-OCTEVILLE', 
                    capacity=self.size, level=self.size,
                    loc=shapely.geometry.Point(0, 0))
        port = Site(**port)

        owf = dict(env=self.env, name='SAINT BRIEUC OFFSHORE WIND FARM', 
                   capacity=self.size, level=0,
                   loc=shapely.geometry.Point(0, 0))
        
        owf = Site(**owf)

        aeolus = dict(env=self.env, name='Aeolus', capacity=4, velocity=10, 
                      loc=shapely.geometry.Point(0, 0), port=port, owf=owf)

        aeolus = InstallationVessel(**aeolus)

    def start_simulation(self):

        for obj in InstallationVessel.instances:
            obj.execute_activities()

        end = datetime.datetime(2021, 1, 2).timestamp()
        self.env.run()

        print(pd.DataFrame(obj.log))


# ----------------------------------------------------------------------------!
class Site(core.Identifiable, core.Log, core.Locatable, core.HasContainer,
           core.HasResource, core.Processor, core.LoadingFunction,
           core.UnloadingFunction):
    '''
    --------------------------------------------------------------------------
    Description:
        Python object used for describing the sites involved in constructing
        the offshore wind farm, such as a port and the offshore area. Contains
        information on the number of components in stock and installed, but 
        also their specific location. 
    --------------------------------------------------------------------------
    Parameters:
        env: simpy.Environment
            A simpy environment in which the simulation is carried out.
        name: str
            Identification of the site. 
        loc: shapely.geometry.Point
            The lon, lat position of the site passed through with the 
            shapely.geometry.Point class.
    --------------------------------------------------------------------------
    '''

    instances = []

    def __init__(self, env: simpy.Environment, name: str, capacity: int, 
                 level: int, loc: shapely.geometry.Point, **kwargs):
        
        # Assign arguments to class attributes
        self.env = env
        self.name = name
        self.capacity = capacity
        self.level = level
        self.geometry = loc
        self.loading_rate = 1
        self.unloading_rate = 1

        # Pass arguments to parent classes
        super().__init__(**self.__dict__, **kwargs)

        Site.instances.append(self)


# ----------------------------------------------------------------------------!
class InstallationVessel(core.Identifiable, core.Log,
                         core.ContainerDependentMovable, core.Processor, 
                         core.HasContainer, core.HasResource, 
                         core.LoadingFunction, core.UnloadingFunction):

    '''
    --------------------------------------------------------------------------
    Description:
        Python object used for describing the vessels used to install the 
        components and its activities. The class contains information on 
        vessel characterisitics, etc.
    --------------------------------------------------------------------------
    Parameters:
        env: simpy.Environment
            A simpy environment in which the simulation is carried out.
        name: str
            Identification of the vessel.
        capacity: int
            Number of components allowed on deck.
        velocity: float
            Velocity at which the ship cruises. 

    --------------------------------------------------------------------------
    '''

    instances = []

    def __init__(self, env: simpy.Environment, name: str, capacity: int,
                 loc: shapely.geometry.Point, velocity: float, port: Site,
                 owf: Site, **kwargs):

        # Assign arguments to class attributes
        self.env = env
        self.name = name
        self.capacity = capacity
        self.level = 0
        self.geometry = loc
        self.compute_v = velocity
        self.loading_rate = 1
        self.unloading_rate = 1

        super().__init__(**self.__dict__, **kwargs)

        self.port = port
        self.owf = owf

        InstallationVessel.instances.append(self)

    def execute_activities(self):
        
        # Navigate from offshore to port activitiy
        navigate_to = Navigate(env=self.env, name=self.name, mover=self,
                               destination=self.port, length=3600,
                               ID='Sail from site to port',
                               postpone_start=True)

        # Navigate from port to site activitiy
        navigate_from = Navigate(env=self.env, name=self.name, mover=self,
                                 destination=self.owf, length=3600,
                                 ID='Sail from port to site',
                                 postpone_start=True)

        # Navigate from site to site activitiy
        navigate_between = Navigate(env=self.env, name=self.name, mover=self,
                                    destination=self.owf, length=3600,
                                    ID='Sail from site to site',
                                    postpone_start=True)

        # Load component on vessel activity
        activity = dict(env=self.env, name=self.name, processor=self.port,
                        origin=self.port, destination=self, length=3600,
                        ID='Load component onto deck', postpone_start=True)
        
        load_vessel = LoadComponent(**activity)
        
        expression = [{ 'or': [{
                'type': 'container',
                'concept': self.port,
                'state': 'empty'
            },
            {
                'type': 'container',
                'concept': self,
                'state': 'full'
            }]
        }]

        load_cycle = model.WhileActivity(env=self.env, name=self.name,
                                         registry={},
                                         sub_processes=[load_vessel], 
                                         condition_event=expression,
                                         postpone_start=True)

        # Install and unload component
        activity = dict(env=self.env, name=self.name, processor=self,
                        origin=self, destination=self.owf, length=3600,
                        ID='Install component onto position', 
                        postpone_start=True)
        
        install = InstallComponent(**activity)
        
        expr = [{
            'type': 'container',
            'concept': self,
            'state': 'empty'
        }]

        load_cycle = model.WhileActivity(env=self.env, name=self.name,
                                         registry={},
                                         sub_processes=[install,
                                                        navigate_between], 
                                         condition_event=expr,
                                         postpone_start=True)
        

        # Main installation cycle
        main_cycle = [navigate_to, load_cycle, navigate_from]
        main_cycle = model.SequentialActivity(env=self.env, name=self.name,
                                              registry={},
                                              sub_processes=main_cycle)

# ----------------------------------------------------------------------------!
class Navigate(model.MoveActivity): 
    '''
    --------------------------------------------------------------------------
    Description:
        Python object used for describing the installation process
        'navigating'.
    --------------------------------------------------------------------------
    '''

    def __init__(self, env: simpy.Environment, name: str, 
                 mover: InstallationVessel, destination: Site, length: int, 
                 **kwargs):
        
        self.env = env
        self.name = name
        self.registry = {}
        self.mover = mover
        self.destination = destination
        self.duration = length

        super().__init__(**self.__dict__, **kwargs)


# ----------------------------------------------------------------------------!
class LoadComponent(model.ShiftAmountActivity):
    '''
    --------------------------------------------------------------------------
    Description:
        Python object used for describing the installation process in which
        turbine components are loaded onto a vessel.
    --------------------------------------------------------------------------
    '''
    def __init__(self, env: simpy.Environment, name: str, processor: Site, 
                 origin: Site, destination: InstallationVessel, length: int,
                 amount: int = 1, postpone_start: bool = False, **kwargs):
        
        self.env = env
        self.name = name
        self.registry = {}
        self.processor = processor
        self.origin = origin
        self.destination = destination
        self.amount = amount
        self.duration = length
        self.postpone_start = postpone_start
    
        super().__init__(**self.__dict__, **kwargs)


# ----------------------------------------------------------------------------!
class InstallComponent(model.ShiftAmountActivity):
    '''
    --------------------------------------------------------------------------
    Description:
        Python object used for describing the installation process in which
        turbine components are installed and thus unloaded onto site.
    --------------------------------------------------------------------------
    '''
    def __init__(self, env: simpy.Environment, name: str, processor: Site, 
                 origin: Site, destination: InstallationVessel, length: int,
                 amount: int = 1, postpone_start: bool = False, **kwargs):
        
        self.env = env
        self.name = name
        self.registry = {}
        self.processor = processor
        self.origin = origin
        self.destination = destination
        self.amount = amount
        self.duration = length
        self.postpone_start = postpone_start
    
        super().__init__(**self.__dict__, **kwargs)

In [38]:
st_brieuc = dict(start_date=datetime.datetime(2021, 1, 1), size=10)
st_brieuc = OffshoreWindFarmInstallation(**st_brieuc)
st_brieuc.create_entities()
st_brieuc.start_simulation()

RuntimeError: Attempting to shift content from an empty origin or to a full self.destination. ({'origin.64428f13-01b5-43f1-abc9-69bdca2d172f': 0, 'destination.7a5779f9-9b24-44dc-996c-3f78cdfa1460': 10})